In [ ]:
#Importamos librerías
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import unicodedata
import ftfy
import difflib
from scipy.stats import ks_2samp, mannwhitneyu, chi2_contingency
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Cargamos los dataset
inv = pd.read_csv("inventario_central_v2.csv")
trx = pd.read_csv("transacciones_logistica_v2.csv")
fb  = pd.read_csv("feedback_clientes_v2.csv")


In [ ]:
# Función para auditar los dataset
def audit(df):
    return pd.DataFrame({
        "nulos_%": df.isna().mean() * 100,
        "tipo": df.dtypes
    })

## Auditoría de Calidad y Transparencia — Dataset Inventario (inv)


In [ ]:
display(inv.head())

,SKU_ID,Categoria,Stock_Actual,Costo_Unitario_USD,Punto_Reorden,Lead_Time_Dias,Bodega_Origen,Ultima_Revision
0,PROD-1000,smart-phone,NaN,870.38,259,25-30 días,Norte,2025-11-17
1,PROD-1001,Accesorios,476.0,1397.26,169,25-30 días,Norte,2024-03-05
2,PROD-1002,Monitores,1209.0,611.62,214,5,Sur,2024-06-21
3,PROD-1003,smart-phone,1825.0,145.94,187,10,Sur,2025-01-07
4,PROD-1004,Smartphones,1713.0,77.78,105,5,Sur,2024-07-04


In [ ]:
audit(inv)

,nulos_%,tipo
SKU_ID,0.00,object
Categoria,0.00,object
Stock_Actual,4.00,float64
Costo_Unitario_USD,0.00,float64
Punto_Reorden,0.00,int64
Lead_Time_Dias,16.12,object
Bodega_Origen,0.00,object
Ultima_Revision,0.00,object


### FUNCIONES DE LIMPIEZA Y NORMALIZACIÓN

In [ ]:
# Función realizada por IA
def parse_lead_time(x):
    """
    Convierte Lead_Time_Dias a numérico:
    - 'Inmediato' -> 1
    - '25-30 días' -> 30 (toma el valor mayor del rango)
    - '3', '5', '10' -> float
    - NaN se mantiene como NaN
    """
    if pd.isna(x):
        return np.nan

    s = str(x).strip().lower()

    if s in {"inmediato", "inmediate", "immediate"}:
        return 1.0

    # Rango tipo "25-30 días" o "25 - 30"
    m = re.search(r"(\d+(?:\.\d+)?)\s*-\s*(\d+(?:\.\d+)?)", s)
    if m:
        a, b = float(m.group(1)), float(m.group(2))
        return max(a, b)

    # Primer número que aparezca
    m = re.search(r"(\d+(?:\.\d+)?)", s)
    if m:
        return float(m.group(1))

    return np.nan


In [ ]:
# Función realizada por IA para estandarizar la columna categoría
def parse_categoria(x):
    if pd.isna(x):
        return np.nan

    s = str(x).strip().lower()

    # Normalizar separadores
    s = re.sub(r"[_\-\/]+", " ", s)

    # Quitar caracteres no alfanuméricos
    s = re.sub(r"[^a-z0-9\s]", "", s)

    # Colapsar espacios
    s = re.sub(r"\s+", " ", s).strip()

    # Si quedó vacío (ej. "???", "---", "***") → NaN
    if s == "":
        return np.nan

    # Canonicalización semántica
    if "smart" in s and "phone" in s:
        return "Smartphones"

    if s in {"laptop", "laptops"}:
        return "Laptops"

    return s.title()



In [ ]:

## Función realiza por IA para verifcar si hay más de dos filas con datos nulos dados que no se podría imputar
def drop_rows_with_many_nulls(df, k=2):
    """
    Elimina filas que tengan k o más valores nulos.
    Retorna:
      - df_limpio
      - df_eliminados (con las columnas que causaron la eliminación)
    """
    # Contar nulos por fila
    null_count = df.isna().sum(axis=1)

    # Filas a eliminar
    mask_drop = null_count >= k

    # Identificar qué columnas están nulas en esas filas
    cols_nulas = (
        df[mask_drop]
        .isna()
        .apply(lambda row: list(df.columns[row]), axis=1)
    )

    df_eliminados = df[mask_drop].copy()
    df_eliminados["Columnas_Nulas"] = cols_nulas
    df_eliminados["N_Nulos"] = null_count[mask_drop]

    # Dataset limpio
    df_limpio = df[~mask_drop].copy()

    return df_limpio, df_eliminados


### NORMALIZACION DE CAMPOS (Lead_Time_Dias, Categoria)

Para el caso de la columna Lead_Time_Dias, se observa valores como inmediatos o  25 - 30 días, para este caso, si es inmediato se reemplazara por un 1, indicando que se vende en el mismo día, y para el otro caso, se deja el valor más alto para tener un margen

Se estandariza la columna categoría, para evitar errores como lo son al momento de realizar las graficas, para que tome un uico registro, es decir, en ligar de smart-phone, sea smarphones

In [ ]:
inv["Lead_Time_Dias"] = inv["Lead_Time_Dias"].apply(parse_lead_time)
inv["Categoria"] = inv["Categoria"].apply(parse_categoria)


In [ ]:
# Validación
display(inv[["Lead_Time_Dias", "Categoria"]].head())
print("Categorías únicas (muestra):", inv["Categoria"].dropna().unique()[:15])

,Lead_Time_Dias,Categoria
0,30.0,Smartphones
1,30.0,Accesorios
2,5.0,Monitores
3,10.0,Smartphones
4,5.0,Smartphones


Categorías únicas (muestra): ['Smartphones' 'Accesorios' 'Monitores' 'Tablets' 'Laptops']



### FILTRADO DE FILAS CON NULOS (k ≥ 2)



Como se debe de imputar loS valores de las columnas, primero verificamos cuales registros tienen mas de dos columnas nulas, por lo que la imputación sería más completas y se eliminaran para evitar que estos registros nos afecten análisis posteriores

In [ ]:
inv_limpio, inv_eliminados = drop_rows_with_many_nulls(inv, k=2)
print("Filas eliminadas:", len(inv_eliminados))
print("Filas restantes:", len(inv_limpio))


Filas eliminadas: 87
Filas restantes: 2413


### IMPUTACION 1: Lead_Time_Dias

Ahora, imputamos los valores nulos de Lead_Time_Dia con la mediana

In [ ]:
mediana_lt = inv_limpio["Lead_Time_Dias"].median()
n_nulos_antes = inv_limpio["Lead_Time_Dias"].isna().sum()

inv_limpio["Lead_Time_Dias"] = inv_limpio["Lead_Time_Dias"].fillna(mediana_lt)

n_nulos_despues = inv_limpio["Lead_Time_Dias"].isna().sum()
valores_finales = np.sort(inv_limpio["Lead_Time_Dias"].unique())

print(f"Mediana usada: {mediana_lt}")
print(f"Valores imputados: {n_nulos_antes}")
print(f"Nulos restantes: {n_nulos_despues}")
print(f"Valores únicos finales (Lead_Time_Dias):\n{valores_finales}")

Mediana usada: 5.0
Valores imputados: 323
Nulos restantes: 0
Valores únicos finales (Lead_Time_Dias):
[ 1.  3.  5. 10. 30.]


### NORMALIZACION Y ANALISIS DE BODEGA_ORIGEN

In [ ]:
inv_limpio["Bodega_Origen"].unique()

array(['Norte', 'Sur', 'ZONA_FRANCA', 'norte', 'BOD-EXT-99', 'Occidente'],
      dtype=object)

In [ ]:
valores_bodega_origen = inv_limpio["Bodega_Origen"].dropna().unique()

print(f"Valores únicos en Bodega_Origen ({len(valores_bodega_origen)}):")
print(valores_bodega_origen)


Valores únicos en Bodega_Origen (6):
['Norte' 'Sur' 'ZONA_FRANCA' 'norte' 'BOD-EXT-99' 'Occidente']


In [ ]:
# Normalizar solo los valores específicos
inv_limpio["Bodega_Origen"] = inv_limpio["Bodega_Origen"].replace({
    "norte": "Norte",
    "ZONA_FRANCA": "Zona Franca"
})

valores_bodega_origen = np.sort(inv_limpio["Bodega_Origen"].dropna().unique())

print(f"Valores únicos en Bodega_Origen ({len(valores_bodega_origen)}):")
print(valores_bodega_origen)

Valores únicos en Bodega_Origen (5):
['BOD-EXT-99' 'Norte' 'Occidente' 'Sur' 'Zona Franca']


In [ ]:
df_lt = inv_limpio[inv_limpio["Lead_Time_Dias"].notna()].copy()

bod_ext = df_lt[df_lt["Bodega_Origen"] == "BOD-EXT-99"]
zonas = df_lt[df_lt["Bodega_Origen"].isin(["Norte", "Sur", "Occidente", "Zona Franca"])]

# Estadísticas por zona
stats_zonas = zonas.groupby("Bodega_Origen")["Lead_Time_Dias"].agg(
    n="count",
    media="mean",
    mediana="median",
    std="std"
)

# Estadísticas de BOD-EXT-99
stats_bod = bod_ext["Lead_Time_Dias"].agg(
    n="count",
    media="mean",
    mediana="median",
    std="std"
)

print("Estadísticas por zona")
print(stats_zonas)

print("\nEstadísticas BOD-EXT-99")
print(stats_bod)

# Comparar distancia de medianas
distancias = (stats_zonas["mediana"] - stats_bod["mediana"]).abs()

print("\nDistancia de medianas contra BOD-EXT-99:")
print(distancias)



Estadísticas por zona
                 n      media  mediana        std
Bodega_Origen                                    
Norte          813   9.460025      5.0  10.166953
Occidente      372  10.293011      5.0  10.758694
Sur            422   9.097156      5.0  10.107383
Zona Franca    394   9.076142      5.0   9.845850

Estadísticas BOD-EXT-99
n          412.000000
media        9.771845
mediana      5.000000
std         10.399859
Name: Lead_Time_Dias, dtype: float64

Distancia de medianas contra BOD-EXT-99:
Bodega_Origen
Norte          0.0
Occidente      0.0
Sur            0.0
Zona Franca    0.0
Name: mediana, dtype: float64


BOD-EXT-99 se comporta como una bodega promedio, pero sigue siendo una bodega externa. Por lo tanto, será una nueva bodega

In [ ]:
# Normalizar BOD-EXT-99
inv_limpio["Bodega_Origen"] = inv_limpio["Bodega_Origen"].replace({
    "BOD-EXT-99": "Externa"
})


### IMPUTACION 2: Stock_Actual por mediana dentro de Categoria

Para imputar Stock_Actual, se debe hacer de forma condicional por grupo, no una imputación global. Por lo tanto se agrupa las categorías y luego con la mediana en la columna Stock_Actual, lo que nos ayuda

In [ ]:
# Calcular mediana de Stock_Actual por Categoría
medianas_por_categoria = inv_limpio.groupby("Categoria")["Stock_Actual"].median()

# Contar cuántos valores nulos hay antes
n_nulos_antes = inv_limpio["Stock_Actual"].isna().sum()

# Imputar usando la mediana de su propia categoría
inv_limpio["Stock_Actual"] = inv_limpio.apply(
    lambda row: medianas_por_categoria[row["Categoria"]]
                if pd.isna(row["Stock_Actual"])
                else row["Stock_Actual"],
    axis=1
)

# Contar cuántos quedan después
n_nulos_despues = inv_limpio["Stock_Actual"].isna().sum()

# Mostrar resultados
print("Medianas por Categoría:")
print(medianas_por_categoria)

print(f"\nValores imputados: {n_nulos_antes}")
print(f"Nulos restantes después de imputar: {n_nulos_despues}")


Medianas por Categoría:
Categoria
Accesorios      977.5
Laptops         940.0
Monitores       940.5
Smartphones    1041.0
Tablets         986.0
Name: Stock_Actual, dtype: float64

Valores imputados: 72
Nulos restantes después de imputar: 0


In [ ]:
audit(inv_limpio)

,nulos_%,tipo
SKU_ID,0.000000,object
Categoria,9.697472,object
Stock_Actual,0.000000,float64
Costo_Unitario_USD,0.000000,float64
Punto_Reorden,0.000000,int64
Lead_Time_Dias,0.000000,float64
Bodega_Origen,0.000000,object
Ultima_Revision,0.000000,object


Se imputa la Categoria usando Bodega_Origen y Stock_Actual porque el análisis mostró que el Lead Time no discrimina (la mediana es 5 días para todas las combinaciones), mientras que el nivel de inventario sí diferencia claramente las categorías dentro de cada bodega; por eso, para una fila sin categoría, se busca dentro de su misma bodega la categoría cuya mediana de stock sea más cercana a su Stock_Actual, asignando la categoría más compatible con su perfil operativo real, lo que preserva la lógica logística y evita introducir sesgos artificiales.

### IMPUTACION 3: Categoria faltante usando perfil de stock por bodega

In [ ]:
nulos_antes = inv_limpio["Categoria"].isna().sum()
print("Nulos en Categoria (antes):", nulos_antes)

Nulos en Categoria (antes): 234


In [ ]:
perfil = inv_limpio.dropna(subset=["Categoria"]).groupby(
    ["Bodega_Origen", "Categoria"]
).agg(
    stock_mediana=("Stock_Actual", "median"),
    n=("Categoria", "count")
).reset_index()


In [ ]:
display(perfil.head(10))

,Bodega_Origen,Categoria,stock_mediana,n
0,Externa,Accesorios,977.5,50
1,Externa,Laptops,954.0,92
2,Externa,Monitores,940.5,63
3,Externa,Smartphones,1041.0,115
4,Externa,Tablets,980.0,47
5,Norte,Accesorios,1072.5,118
6,Norte,Laptops,942.0,221
7,Norte,Monitores,920.0,116
8,Norte,Smartphones,1041.0,188
9,Norte,Tablets,955.0,96


In [ ]:
def inferir_categoria_por_stock(row):
    zona = row["Bodega_Origen"]
    stock = row["Stock_Actual"]

    sub = perfil[perfil["Bodega_Origen"] == zona]

    # Si la bodega no tiene suficiente info, usar todas
    if len(sub) == 0:
        sub = perfil

    distancias = (sub["stock_mediana"] - stock).abs()

    return sub.loc[distancias.idxmin(), "Categoria"]


In [ ]:
nulos_antes = inv_limpio["Categoria"].isna().sum()

inv_limpio.loc[inv_limpio["Categoria"].isna(), "Categoria"] = (
    inv_limpio[inv_limpio["Categoria"].isna()].apply(inferir_categoria_por_stock, axis=1)
)

nulos_despues = inv_limpio["Categoria"].isna().sum()

print(f"Categorías imputadas: {nulos_antes}")
print(f"Nulos restantes: {nulos_despues}")


Categorías imputadas: 234
Nulos restantes: 0


In [ ]:
audit(inv_limpio)

,nulos_%,tipo
SKU_ID,0.0,object
Categoria,0.0,object
Stock_Actual,0.0,float64
Costo_Unitario_USD,0.0,float64
Punto_Reorden,0.0,int64
Lead_Time_Dias,0.0,float64
Bodega_Origen,0.0,object
Ultima_Revision,0.0,object


### EXPORTACIÓN FINAL

In [ ]:
# Guardar el DataFrame como CSV
inv_limpio.to_csv("inventario_central_v2_limpio.csv", index=False, encoding="utf-8")

print("Archivo 'inventario_central_v2_limpio.csv' creado correctamente.")


Archivo 'inventario_central_v2_limpio.csv' creado correctamente.


## Auditoría de Calidad y Transparencia — Dataset Transacciones (trx)

In [ ]:
audit(trx)

,nulos_%,tipo
Transaccion_ID,0.00,object
SKU_ID,0.00,object
Fecha_Venta,0.00,object
Cantidad_Vendida,0.00,int64
Precio_Venta_Final,0.00,float64
Costo_Envio,8.34,float64
Tiempo_Entrega_Real,0.00,int64
Estado_Envio,16.83,object
Ciudad_Destino,0.00,object
Canal_Venta,0.00,object


### FUNCIONES DE LIMPIEZA Y NORMALIZACIÓN

In [ ]:
def normalize_text_full(s):
    if pd.isna(s): return s
    s = ftfy.fix_text(s)
    s = s.lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = re.sub(r"[^a-z0-9\s]", "", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def build_fuzzy_map(values, threshold=0.9):
    values = sorted(set(values.dropna()))
    canonical, mapping = [], {}
    for v in values:
        match = difflib.get_close_matches(v, canonical, n=1, cutoff=threshold)
        if match: mapping[v] = match[0]
        else:
            canonical.append(v)
            mapping[v] = v
    return mapping

def clean_numeric_outliers(df, numeric_cols, iqr_factor=1.5):
    df_clean = df.copy()
    for col in numeric_cols:
        series = df_clean[col]
        if series.dropna().empty: continue
        q1, q3 = series.quantile(0.25), series.quantile(0.75)
        iqr = q3 - q1
        lower, upper = q1 - iqr_factor * iqr, q3 + iqr_factor * iqr
        df_clean[col] = series.clip(lower, upper).fillna(series.median())
    return df_clean

### PROCESAMIENTO DE TRANSACCIONES

In [ ]:
trx["Cantidad_Vendida"] = trx["Cantidad_Vendida"].replace(-5, 5)
trx['Tiempo_Entrega_Real'] = trx['Tiempo_Entrega_Real'].replace(999, np.nan)

# Normalización de fechas
TODAY = pd.Timestamp("2026-01-31")
trx["Fecha_Venta"] = pd.to_datetime(trx["Fecha_Venta"], errors="coerce")
trx.loc[trx["Fecha_Venta"] > TODAY, "Fecha_Venta"] = pd.NaT

# Limpieza de strings vacíos o nulos disfrazados
EMPTY_VALUES = ["", " ", "nan", "NaN", "null", "NULL", "none", "None", "?", "-", "--"]
trx = trx.replace(EMPTY_VALUES, np.nan).replace(r"^\s*$", np.nan, regex=True)

/tmp/ipython-input-1510342991.py:6: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  trx["Fecha_Venta"] = pd.to_datetime(trx["Fecha_Venta"], errors="coerce")


### NORMALIZACIÓN DE TEXTO (Ciudades y Canales)

In [ ]:
city_aliases = {"med": "medellin", "mde": "medellin", "medell": "medellin", "bog": "bogota", "bta": "bogota", "bgta": "bogota"}

for col in ["Ciudad_Destino", "Canal_Venta"]:
    trx[f"{col}_norm"] = trx[col].apply(normalize_text_full)

trx["Ciudad_Destino_norm"] = trx["Ciudad_Destino_norm"].replace(city_aliases)

# Aplicar mapeo difuso (Fuzzy Matching)
city_map = build_fuzzy_map(trx["Ciudad_Destino_norm"], threshold=0.9)
trx["Ciudad_Destino_norm"] = trx["Ciudad_Destino_norm"].map(city_map)

### IMPUTACIÓN DE DATOS

In [ ]:
trx = trx[trx.isna().sum(axis=1) < 2].copy()

# Imputación Numérica Avanzada (Iterative Imputer)
numeric_cols = trx.select_dtypes(include="number").columns
it_imputer = IterativeImputer(max_iter=10, random_state=42)
trx[numeric_cols] = it_imputer.fit_transform(trx[numeric_cols])

# Winsorización de Outliers
trx = clean_numeric_outliers(trx, numeric_cols)

# Imputación Categórica (Random Forest para 'Estado_Envio')
col_obj = 'Estado_Envio'
cols_base = ['Cantidad_Vendida', 'Precio_Venta_Final', 'Costo_Envio', 'Tiempo_Entrega_Real']

if trx[col_obj].isna().any():
    imp_simple = SimpleImputer(strategy='median')
    X_val = pd.DataFrame(imp_simple.fit_transform(trx[cols_base]), index=trx.index)

    train_idx = trx[trx[col_obj].notna()].index
    predict_idx = trx[trx[col_obj].isna()].index

    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_val.loc[train_idx], trx.loc[train_idx, col_obj])

    trx.loc[predict_idx, col_obj] = rf_model.predict(X_val.loc[predict_idx])
    print(f"Imputados {len(predict_idx)} valores en {col_obj}")

Imputados 1516 valores en Estado_Envio


### EXPORTACIÓN FINAL

In [ ]:

trx.to_csv("transacciones_logistica_final_unificado.csv", index=False)
print("Proceso completado. Archivo guardado con éxito.")


Proceso completado. Archivo guardado con éxito.


## Auditoría de Calidad y Transparencia — Dataset Feedback (fb)

In [ ]:
audit(fb)

,nulos_%,tipo
Feedback_ID,0.000000,object
Transaccion_ID,0.000000,object
Rating_Producto,0.000000,int64
Rating_Logistica,0.000000,int64
Comentario_Texto,14.600000,object
Recomienda_Marca,24.866667,object
Ticket_Soporte_Abierto,0.000000,object
Edad_Cliente,0.000000,int64
Satisfaccion_NPS,0.000000,float64
